<a href="https://colab.research.google.com/github/DanRHowarth/Artificial-Intelligence-Cloud-and-Edge-Implementations/blob/master/Oxford_End_to_End_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to End ML Problem: Classification

* This is a `binary classification` problem, as we are training a model to differentiate between two different target classes, or categories. This is different from a `mutli-class classification` problem, which has more than two classes. The principles are the same, but the algorithms we use and the evaluation metrics can be different deoending on the type of problem.
* We will tackle this problem using the same ML workflow as before; we will introduce you to the key libraries and algorithms; and we will provide code that you can use yourself. So let's begin wiht a recap of the `machine learning workflow` before moving on to the libraries we will use and the problem we will solve. 

## RECAP: Machine Learning Workflow

![alt text](https://github.com/DanRHowarth/Artificial-Intelligence-Cloud-and-Edge-Implementations/blob/master/Screenshot%202019-11-11%20at%2022.01.44.png?raw=true)

### 1. Define the Problem Statement

### 2. Analyze and Preprocess Data

### 3. Split the data set 

### 4. Choose the most appropriate baseline algorithm

### 5. Train and test your baseline model

### 6. Chose quality evaluation metric(s)

### 7. Refine our dataset to improve the baseline model

### 8. Test alternative models

### 9. Choose the best model and optimize it's parameters

## RECAP: Key `python` libraries 

* We will use the following `python` libraries, which you will encounter frequently for data analysis and machine learning tasks: 
 *  `numpy`, which provides vectorised arrays, and maths, algebra functionality;
 * `pandas`, which provides data structures and data analysis tools;
 * `matplotlib`, which provides highly customisable plotting functionality (and we also use `seaborn`, built on top of `matplotlib`, which is less customisable but can generate charts with less code); and, 
 * `scikit-learn`, which provides models and tools for most machine learning algorithms

In [0]:
# import main data analysis libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# note we use scipy for generating a uniform distribution in the model optimization step
from scipy.stats import uniform

# note that because of the different dataset and algorithms, we use different sklearn libraries from Day 1 
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# hide warnings
import warnings
warnings.filterwarnings('ignore')

## Machine Learning Workflow in Action
* To take you through a machine learning problem following the steps introduced earlier 
* To explain at a high level the different steps, and the code used to implement them
* To introduce new concepts as and when required

### 1. Define the Problem Statement
* For our second end to end problem, we will use anoterh dataset that is provided as part of the `sci-kit learn` library, the breast cancer dataset. We can load this dataset easily and see a general description of what it contains.

In [0]:
# we load the dataset and save it as the variable data
data = load_breast_cancer()

# if we want to know what sort of detail is provided with this dataset, we can call .keys()
data.keys()

In [0]:
# the info at the .DESCR key will tell us more 
print (data.DESCR)

* There is a lot of information to inform our problem statement. We have been given a `data` and `target` set of values and told that the target values are two classes - Malignant and Benign. We also know there are 212 Malignant and 357 Benign values and associated features. 
* We can therefore say that the problem statement is *to predict the class given a set number of features*. 
* We have 30  features. We will use those eatures and associated target variable  to train a model using seen or training data. We will then feed our trained model the 30 features from unseen or test data *without* passing in the target variable information. Our model will predict a class based on these features and we will compare that prediction to the actual answer to assess how well our model performs. 
* We can also see that our data set contains 569 records. It has no missing values, which is unusual but makes our job of `preprocessing` easier.

### Step 2: Analyse and Preprocess data

#### Analyze the Data
* We will use `pandas` and `matplotlib` to do some basic `exploratory data analysis`
* This will include getting a feel for the overall dataset so that we understand what sorts of values it contains
* We will compute summary statistics and look at the distributions of each feature

In [0]:
# we can use pandas to create a dataframe, which is basically a way of storing and operating on tabular data 
# here we pass in both the data and the column names as variables
X = pd.DataFrame(data.data, columns = data.feature_names)

In [0]:
# we can then look at the top of the dataframe to see the sort of values it contains
X.describe(include = 'all')

In [0]:
# we can now look at our target variable 
y = data.target

# we can see that it is a list of 0s and 1s, with 1s matching to the Benign class
y

In [0]:
# we can analyse the data in more detail by understanding how the features and target variables interact
# we can do this by grouping the features and the target into the same dataframe
# note we create a copy of the data here so that any changes don't impact the original data

full_dataset = X.copy()
full_dataset['target'] = y.copy()

In [0]:
# let's take a look at the first few lines of the dataset
full_dataset.head()

In [0]:
# lets see how balanced the classes are (and if that matches to our expectation)
full_dataset['target'].value_counts()

In [0]:
# let's evaluate visually how well our classes are differentiable on the pairplots
# can see two classes being present on a two variables charts?
# the pairplot function is an excellent way of seeing how variables inter-relate, but 30 feature can make studying each combination difficult!
sns.pairplot(full_dataset, hue = 'target')

* We can clearly see the presence of two clouds with different colors, representing our target classes. 
* Of course, they are still mixed to some extent, but if we were to visualise the variables in multi-dimentional space they would become more separable.
* Now let's check the Pearson's correlation between pairs of our features and also between the features and our target. 

In [0]:
# we can again use seaborn to easily create a visually interesting chart 
plt.figure(figsize = (15, 10))

# we can add the annot=True parameter to the sns.heatmap arguments if we want to show the correlation values 
sns.heatmap(full_dataset.corr(method='pearson'))

* Dark red colours are positilvey correlated with the corresponding feature, dark blue features are negatively correlated.
* We can see that some values are negatively correlated with our target variable.
* This information could help us with feature engineering.

###  Step 3: Split the data
* In order to train our model and see how well it performs, we need to split our data into training and testing sets.
* We can then train our model on the training set, and test how well it has generalised to the data on the test set.
* There are a number of options for how we can split the data, and for what proportion of our original data we set aside for the test set.

In [0]:
# Because our classes are not absolutely equal in number, we can apply stratification to the split 
# and be sure that the ratio of the classes in both train and test will be the same 
# you can learn about the other parameters by looking at the documentation 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle=True)

In [0]:
# as with Day 1, we can get shape of test and training sets
print('Training Set:')
print('Number of datapoints: ', X_train.shape[0])
print('Number of features: ', X_train.shape[1])
print('\n')
print('Test Set:')
print('Number of datapoints: ', X_test.shape[0])
print('Number of features: ', X_test.shape[1])

In [0]:
# and we can verify the stratifications using np.bincount
print('Labels counts in y:', np.bincount(y))
print('Percentage of class zeroes in class_y',np.round(np.bincount(y)[0]/len(y)*100))

print("\n")
print('Labels counts in y_train:', np.bincount(y_train))
print('Percentage of class zeroes in y_train',np.round(np.bincount(y_train)[0]/len(y_train)*100))

print("\n")
print('Labels counts in y_test:', np.bincount(y_test))
print('Percentage of class zeroes in y_test',np.round(np.bincount(y_test)[0]/len(y_test)*100))

###  Step 4: Choose a Baseline algorithm
* Building a model in `sklearn` involves:
  * defining / instantiating the model we want to use and its parameters (**Step 4**)
  * fitting the model we have developed to our training set (**Step 5**)
* We can then use the model to predict scores against our test set and assess how good it is
* To do this, we need to define an evaluation metric (**Step 6**). There are a number of different options, and they differ for both regression and classification problems. This score will be what we use to select our best model, and the best parameters. 
* We will take through these steps now. As you will see, the code required to implement these steps is minimal, thanks to different methods provided for us by  `sklearn`.

In [0]:
## we can create a baseline model to benchmark our other estimators against
## this can be a simple estimator or we can use a dummy estimator to make predictions in a random manner 

# this creates our dummy classifier, and the value we pass in to the strategy parameter dtermn
dummy = DummyClassifier(strategy='uniform', random_state=1)

### Step 5: Train and Test the Model 

In [0]:
# "Train" model
dummy.fit(X_train, y_train)

# from this, we can generate a set of predictions on our unseen features, X_test
dummy_predictions = dummy.predict(X_test)

###  Step 6: Choose an evaluation metric
* We then need to compare these predictions with the actual result and measure them in some way. This is where the selection of evaluation metric is important. 
* Classification metrics include:
  * `accuracy`: this assess how often the model selects the best class. This can be more useful when there are balanced classes (i.e. there are a similar number of instances of each class we are trying to predict). 
    * There are some limits to this metric. For example, if we think about something like credit card fraud, where the instances of fraudulent transactions might be 0.5%, then a model that *always* predicts that a transaction is not fraudulent will be 99.5% accurate! So we often need metrics that can tell us how a model performs in more detail. 
  * `f1 score`: 
  * `roc_auc`: 
  * `recall`: 
  * We recommend you research these metrics to improve your understanding of how they work. Try to look up an explanation or two (for example on wikipedia and scikit-learn documentation) and write a one line summary in the space provided above. Then, below, when we implement a scoring function, select these different metrics and try to explain what is happening. This will help cement you knowledge.

In [0]:
def evaluate(y_test, y_pred):
    # this block of code returns all the metrics we are interested in 
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    auc = metrics.roc_auc_score(y_test, y_pred)

    print ("Accuracy", accuracy)
    print ('F1 score: ', f1)
    print ('ROC_AUC: ' , auc)

In [0]:
# we can call the function on the actual results versus the predictions
# we will see that the metrics are what we'd expect from a random model 
evaluate(y_test, dummy_predictions)

### Step 7: Refine our dataset
* In Day 1, we undertook some featuring engineering to try and make the dataset more representative of the problem we were trying to solve.
* We will focus on other steps today, but please feel free to try this step in order to build your understanding of this technique

In [0]:
# FEATURE ENGINEERING CODE HERE



### Step 8: Test Alternative Models
* Once we got a nice baseline model working for this dataset, we also can try something more sophisticated and rather different. 
* As well as using a different estimator, we can use a different method, `cross_validation`, to generate a score. This is shown to broaden your knowledge.
  * There are a number of different ways in `scikit-learn` to get an estimator score and it can get confusing first.
  * Remember that to get a score, we need to instantiate a model, fit it to the data, predict using unseen data, compare the predictions against actual data, and score the difference. This is true for classification and regression problems, and is true no matter the method used to get there.
    * So, in the end-to-end tutorials we split the training and test data,  fitted our data to an estimator, and called the `.predict` method on the estimator to get our predictions, and then passed this to a scoring function (four steps)
    * We can use `estimator.score()`method, where we pass in our split data and the method then makes predictions and returning the score (three steps). 
    * And, in the `cross_val_score()` method used below we are effectively using one step as the method takes an estimator and our data and returns a score. You can find out more about this method [here](https://scikit-learn.org/stable/modules/cross_validation.html) 

In [0]:
## here we fit a new estimator and use cross_val_score to get a score based on a defined metric 

# instantiate logistic regression classifier
logistic = LogisticRegression()

# we pass our estimator and data to the method. we also specify the number of folds (default is 3)
# the default scoring method is the one associated with the estimator we pass in
# we can use the scoring parameter to pass in different scoring methods. Here we use f1.  
cross_val_score(logistic, X, y, cv=5, scoring="f1")

In [0]:
# we can see that this returns a score for all the five folds of the cross_validation
# if we want to return a mean, we can store as a variable and calculate the mean, or do it directly on the function
# this time we will use accuracy
cross_val_score(logistic, X, y, cv=5, scoring="accuracy").mean()

In [0]:
# lets do this again with a different model
rnd_clf = RandomForestClassifier()

# and pass that in 
cross_val_score(rnd_clf, X, y, cv=5, scoring="accuracy").mean()

#### Ensemble models 

* Let's take this opportunity to explore ensemble methods.
* The goal of ensemble methods is to combine different classifiers into a meta-classifier that has better generalization performance than each individual classifier alone. 
* There are several different approaches to achieve this, including **majority voting** ensemble methods, which we select the class label that has been predicted by the majority of classifiers.
* The ensemble can be built from different classification algorithms, such as decision trees, support vector machines, logistic regression classifiers, and so on. Alternatively, we can also use the same base classification algorithm, fitting different subsets of the training set. 
* Indeed, Majority voting will work best if the classifiers used are different from each other and/or trained on different datasets (or subsets of the same data) in order for their errors to be uncorrelated. 

In [0]:
# lets instantiate an additional model to make an ensemble of three models
dt_clf = DecisionTreeClassifier()

# and an ensemble of them
voting_clf = VotingClassifier(estimators=[('lr', logistic), ('rf', rnd_clf), ('dc', dt_clf)],                              
                              # here we select soft voting, which returns the argmax of the sum of predicted probabilities
                              voting='soft')

In [0]:
# here we can cycle through the individual estimators 
# for clf in (log_clf, rnd_clf, svm_clf, voting_clf):

for clf in (logistic, rnd_clf, dt_clf, voting_clf):
    
    # fit them to the training data 
    clf.fit(X_train, y_train)
    
    # get a prediction
    y_pred = clf.predict(X_test)
    
    # and print the prediction 
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

* We can see that `voting classifier` in this the case does have a slight edge on the other models (note that this could vary depending on how the data is split at training time).
* This is an interesting approach and one to consider when you are developing your models.

### Step 9: Choose the best model and optimise its parameters
* We can see that we have improved our model as we have added features and trained new models.
* At the point that we feel comfortable with a good model, we can start to tune the parameters of the model.
* There are a number of ways to do this. We applied `GridSearchCV` to identify the best hyperparameters for our models on Day 1.
* There are other methods available to use that don't take the brute force approach of `GridSearchCV`.
* We will cover an implementation of `RandomizedSearchCV` below, and use the exercise for you to implement it on the other datatset.
* We use this method to search over defined hyperparameters, like `GridSearchCV`, however a fixed number of parameters are sampled, as defined by `n_iter` parameter.  

In [0]:
# we will optimise logistics regression 
# we can create hyperparameters as a list, as in type regularization penalty 
penalty = ['l1', 'l2']

# or as a distribution of values to sample from -'C' is the hyperparameter controlling the size of the regularisation penelty 
C = uniform(loc=0, scale=4)

# we need to pass these parameters as a dictionary of {param_name: values}
hyperparameters = dict(C=C, penalty=penalty)

In [0]:
# we instantiate our model
randomizedsearch = RandomizedSearchCV(logistic, hyperparameters, random_state=1, 
                                      n_iter=100, cv=5, verbose=0, n_jobs=-1)

In [0]:
# and fit it to the data 
best_model = randomizedsearch.fit(X, y)

In [0]:
# and we can call this method to return the best parameters the search returned
best_model.best_estimator_

#### Using `cross_val_score` to tune hyperparameters
* You may have noticed that we are unable to evaluate the performance of  `best_model` on unseen data because we have used the dataset for the `RandomizedSearchCV` implementation. 
* We could look to use just our `X_train` and `y_train` dataset in `randomizedsearch`and then evaluate on our test sets. 
* Or we could use `cross_val_score`, as we did above. In this instance, it will create a nested cross-validation set, effectively keeping part of the data unseen so we can test it `k` number of times. We will be therefore be able to return a model score and evaluate the model's performance.
* See section 3.7 of [this](https://sebastianraschka.com/pdf/manuscripts/model-eval.pdf) for more detail (and an excellent discussion of model selection overall).

In [0]:
## here we are effectively running the randomizedsearch function 5 times, each time with some data kept aside for evaluation
## this will take a few minutes to run!
cross_val_score(randomizedsearch, X, y, cv=5, scoring="accuracy").mean()

## Summary and Next Steps:
* Thank you so much for joining these las two days. We hope that you learnt a lot and that you have been excited enough to continue your machine learning journey.
* Don't worry if there are some topics that you didn't fully understand. This is a broad, ever-evolving field. Try to focus on developing some specific knowledge of the areas you need most development in - but remember to also relate it to the broader picture to cement your understanding. 
* Please also leave comments on the page where these notebooks where published and we will try to answer them as soon as we can.
